Flask is a fast, lightweight way to connect your Python scripts to a server. It's a simple and robust framework that can do small tasks (create a microblog, stand up a simple API) or complex ones (Pinterest's API, create a twitter clone).

Flask is called a _microframework_ because it is minimalistic and does not impose an application structure on you. Flask allows you to "plug in" libraries of your choice to add additional functionality. "Heavier" frameworks such as Django typically come with their own libraries (e.g. for database access). This is often convenient, but it takes more time to learn and is not as flexible.

Let's jump in with a simple example. Then, we'll expand it to show what it can do with your models. Flask comes preinstalled with Anaconda. However, if you don't have it you can install it:

```bash
$ pip install -U flask
```

In [ ]:
import flask

Let's discuss how Flask is used to render a website:

1. First, each URL is mapped to a function (called **routing**).

2. When the user visits the URL, the associated function is called.

3. The function returns a string of HTML, which is directly rendered by the browser.

That's it! 

> Using this way of thinking, when visiting a URL you are actually making a function call to a remote computer.

> The URL is the function signature (including parameters), and the function returns a value back to the browser -- in this case a string of HTML. In fact, this is why interfaces for retrieving data online are called "Web APIs" -- they are just function calls disguised as URLs!

<a id="part----hello-world"></a>
### Part 1 - Hello, world

Create a new file called `hello.py` . Type in this code line by line. 

```Python
import flask
app = flask.Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()
```

Three things happen here:
- initialize the app
- use built-in decorators to define what happens on a page
- launch the app

Note that `app` must be defined prior to using the decorators! 

Now launch the file from your command line:

```bash
$ python hello.py
* Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
```
Go to that URL to see your app running on your `localhost`.

Typically, we develop apps locally using this development server. When working, we then upload the files to a web host such as EC2.

<a id="part----arguments-and-styling"></a>
### Part 2 - Arguments and styling

Add the following route underneath the hello() function:

```Python
@app.route('/greet/<name>')
def greet(name):
    '''Say hello to your first parameter'''
    return f"Hello, {name}!"
```
Save and relaunch the app. Now navigate to `http://127.0.0.1:8080/greet/Max`. Your function should respond to that input!

Since the `return` statement sends text to an HTML page, you can style our original `hello()` function with HTML tags:

```Python
@app.route("/")
def hello():
    return '''
    <body>
    <h2> Hello World! </h2>
    </body>
    '''
```
We can also call a function, but let's get into that a little later.

<a id="part----add-in-machine-learning"></a>
### Part 3 - Delete and Re-scaffold
We can use Flask as a way to share and host our machine learning predictions.

Import and initialize the flask app, and launch the server at the bottom. Leave room in the middle to add in your model and routes later on.

```Python
import flask
app = flask.Flask(__name__)

#-------- MODEL GOES HERE -----------#

#-------- ROUTES GO HERE -----------#

if __name__ == '__main__':
    '''Connects to the server'''

    HOST = '127.0.0.1'
    PORT = 4000

    app.run(HOST, PORT)
```
Note that this time we specifed the host and port we want the app to run on.

<a id="part----create-and-train-a-model"></a>
### Part 4 - Create and train a Model

Load in the pipe.pkl and test it out:

```Python
import pickle
import pandas as pd

import flask
app = flask.Flask(__name__)

#-------- MODEL GOES HERE -----------#

pipe = pickle.load(open("pipe.pkl","rb"))
new_data = pd.DataFrame([{'sugar': 2.5, 'sulphates': 0.75, 'alcohol': 11.4, 'grape': 'white'}])
pipe.predict(new_data)

#-------- ROUTES GO HERE -----------#

if __name__ == '__main__':
    '''Connects to the server'''
    HOST = '127.0.0.1'
    PORT = 4000
    app.run(HOST, PORT)

```

<a id="part----make-a-simple-api"></a>
### Part 5 - Make a simple API

Here's the fun part. Now that we have a `PREDICTOR`, we need to get some values to make our predictions.

One way to do this is to get information from the **URL parameters**. These are the part of a URL that come after the `?` and are matched by key:value pairs. For example, if you navigate to:

```
http://localhost:4000/predict?sugar=3&sulphates=1&alcohol=14&grape=red
```

then Flask can retrieve that data for you. Let's write a route to do just that:

```Python
#-------- ROUTES GO HERE -----------#

@app.route('/predict', methods=["GET"])
def predict():
    sugar = flask.request.args['sugar']
    sulphates = flask.request.args['sulphates']
    alcohol = flask.request.args['alcohol']
    grape = flask.request.args['grape']

    data = pd.DataFrame([{
        'sugar': sugar,
        'sulphates': sulphates,
        'alcohol': alcohol,
        'grape': grape}])

    pred = pipe.predict(data)[0]
    results = {'quality': round(pred, 1)}
    return flask.jsonify(results)

```

BOOM! Save the file. Launch your app. You now have a simple API for your model.  

Play with the parameters in the URL. You should get different results for quality

<a id="part----make-a-simple-webform"></a>
### Part 6 - Make a simple webform

Let's create a simple web form to read in the inputs.

Create a file `page.html`:

```html
<!doctype html>
<html>
  <head>
    <title>Wine-O-Matic</title>
  </head>
  <body>

    <form action="http://localhost:4000/result" method="POST">
      <p>Sugar <input type="number" name="sugar" /></p>
      <p>Sulphates <input type="number" name="sulphates" /></p>
      <p>Alcohol <input type="number" name="alcohol" /></p>
      <p>Grape <input type="text" name="grape" /></p>

      <p><input type="submit" value="submit" /></p>
    </form>

  </body>
</html>

```

The two most common HTTP methods are `GET` and `POST`. When your browser visits a URL, it sends a `GET` request. However, when you submit a web form, typically a `POST` request is sent. For a `POST` request, a URL is still accessed. However, the parameters are sent inside the request body instead of as part of the URL (as we saw earlier with the `GET` parameters). Sending parameters in the body allows you to send more data than would fit inside the URL. However, it prevents a user from bookmarking the exact URL call and "hides" the submitted data from the user. (It only hides it from your average user -- the submitted data is actually easy to see using your browser's developer tools.)

Luckily, Flask knows how to read `form` tags in an HTML file that have been `POST`'d to the server.

Add two new decorators in below your first one. 

```Python
#---------- METHOD 2 ----------------#

# This method takes input via an HTML page
@app.route('/page')
def page():
   with open("page.html", 'r') as page:
       return page.read()

@app.route('/result', methods=['POST', 'GET'])
def result():
    '''Gets prediction using the HTML form'''
    if flask.request.method == 'POST':

        inputs = flask.request.form

        sugar = inputs['sugar']
        sulphates = inputs['sulphates']
        alcohol = inputs['alcohol']
        grape = inputs['grape']

        data = pd.DataFrame([{
            'sugar': sugar,
            'sulphates': sulphates,
            'alcohol': alcohol,
            'grape': grape}])

        pred = pipe.predict(data)[0]
        results = {'quality': round(pred, 1)}
        return flask.jsonify(results)
```

Save, close, and relaunch the app. Go to `http://127.0.0.1:4000/page` and type in your inputs.

Both methods should still be there. You can either play with the URL parameters at `/predict` or enter them at `/page`